In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as  mpl

In [2]:
mpl.rcParams[u'font.sans-serif'] = ['simhei']
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
def readData(y,s,c):
    df = pd.read_csv("lvr_land/"+ y +"-s" + s + "/" + c + "_lvr_land_A.csv",encoding='utf-8',low_memory=False)
    return df

In [4]:
def combineDF(df1,df2):
    df = pd.concat([df1,df2],axis = 0)
    return df

In [5]:
def selectFarmland(df):
    
    df = df[["鄉鎮市區","土地移轉總面積(平方公尺)","交易標的","都市土地使用分區","非都市土地使用分區","非都市土地使用編定","總價(元)","單價(元/平方公尺)"]]
    df = df[(df.交易標的 == "房地(土地+建物)")]  
    df_farmland = df[(((df.非都市土地使用分區 == '特定農業區')|(df.非都市土地使用分區 == "一般農業區"))&(df.非都市土地使用編定 =="農牧用地"))]
    #排除都市土地
    df_farmland = df_farmland[df_farmland["都市土地使用分區"].isnull()]
    return df_farmland

In [6]:
def fillNanprice(df_farmland):
    #modify zero or nan price
    df_ok = df_farmland[(df_farmland["單價(元/平方公尺)"].notnull())&(df_farmland["單價(元/平方公尺)"] != "0")]
    df = df_farmland[(df_farmland["單價(元/平方公尺)"].isnull())|(df_farmland["單價(元/平方公尺)"] == "0")]
    df = df[(df['土地移轉總面積(平方公尺)'] != "0")&(df['總價(元)'] != "0")]
    df["單價(元/平方公尺)"] = calculateprice(df['總價(元)'],df['土地移轉總面積(平方公尺)'])
    df_farmland = pd.concat([df_ok,df],axis = 0)
    return df_farmland

In [7]:
def transcationNum(df):
    return df['鄉鎮市區'].count()

In [8]:
def calculateprice(total_price,area):
    return pd.to_numeric(total_price)/pd.to_numeric(area)

In [9]:
def totalcalculate(key): 
    value = sum(key.values())   
    return value

In [10]:
list_cityfilename = ['A','B','C','D','E','F','G','H','I','J','K','M','N','O','P','Q','T','U','V']
list_city = ['台北市','台中市','基隆市','台南市','高雄市','新北市','宜蘭縣','桃園縣','嘉義市','新竹縣','苗栗縣','南投縣','彰化縣','新竹市','雲林縣','嘉義縣','屏東縣','花蓮縣','台東縣']

i=0
taiwan_transacation = {}
for c in list_cityfilename: #every city
    
    s = 4 # from 101-s4 to 107-s2
    town_tran_dist = {}
    for y in range(101,108): #every year
        
        df = pd.DataFrame()
        
        while s <= 4:
            # read data
            df_s =  readData(str(y),str(s),c)
            df = combineDF(df,df_s)

            if y == 107 and s == 2:
                break
            s = s+1
        s=1
        # select farmland
        df_farmland = selectFarmland(df)
        # fill nan price
        df_farmland = fillNanprice(df_farmland)
        #縣市各年交易量
        town_tran_dist[y] = transcationNum(df_farmland)
    #全部縣市各年交易量
    taiwan_transacation[list_city[i]]=town_tran_dist

    i = i+1


In [17]:
#縣市各年交易量排名
print("縣市各年交易量排名:")
table_dict = {}
for y in range(101,108):
    sorted_transcation = sorted(taiwan_transacation.items(),key=lambda s:s[1][y], reverse=True)
    table_dict[y] = sorted_transcation
print("|排名\年",end='')
for i in range(101,108):
    print("|"+str(i),end='')
print("|")
for i in range(0,19):
    print("|"+str(i+1)+"|",end='')
    for y in range(101,108):
        print(table_dict[y][i][0]+"/"+str(table_dict[y][i][1][y])+"|",end="") 
    print("")

縣市各年交易量排名:
|排名\年|101|102|103|104|105|106|107|
|1|宜蘭縣/54|宜蘭縣/283|宜蘭縣/375|宜蘭縣/336|宜蘭縣/222|宜蘭縣/184|宜蘭縣/76|
|2|桃園縣/17|桃園縣/149|桃園縣/180|彰化縣/170|桃園縣/117|桃園縣/124|桃園縣/66|
|3|彰化縣/14|彰化縣/85|彰化縣/163|桃園縣/146|彰化縣/113|彰化縣/106|彰化縣/62|
|4|南投縣/12|花蓮縣/77|花蓮縣/98|新竹縣/86|花蓮縣/76|屏東縣/74|屏東縣/37|
|5|屏東縣/12|南投縣/63|台中市/95|花蓮縣/85|苗栗縣/64|南投縣/55|苗栗縣/32|
|6|花蓮縣/12|雲林縣/58|南投縣/85|南投縣/66|新竹縣/62|花蓮縣/47|南投縣/27|
|7|新竹縣/10|苗栗縣/53|苗栗縣/78|雲林縣/61|屏東縣/58|台南市/46|雲林縣/25|
|8|台中市/5|台中市/52|新竹縣/67|屏東縣/59|台南市/49|新竹縣/45|花蓮縣/25|
|9|嘉義縣/5|台南市/44|雲林縣/48|苗栗縣/58|南投縣/48|雲林縣/43|高雄市/20|
|10|台南市/4|屏東縣/40|屏東縣/48|台中市/57|雲林縣/43|台中市/39|嘉義縣/19|
|11|苗栗縣/2|新竹縣/37|台南市/45|台南市/38|台中市/32|苗栗縣/37|新竹縣/18|
|12|高雄市/1|高雄市/20|高雄市/43|高雄市/35|高雄市/31|高雄市/24|台南市/17|
|13|新北市/1|嘉義縣/19|嘉義縣/24|嘉義縣/26|嘉義縣/18|嘉義縣/19|台中市/15|
|14|新竹市/1|新竹市/8|台東縣/11|台東縣/18|台東縣/15|台東縣/9|新北市/2|
|15|雲林縣/1|台東縣/6|新北市/8|新北市/8|新北市/7|新北市/8|新竹市/2|
|16|台北市/0|新北市/4|新竹市/5|台北市/0|台北市/0|新竹市/3|台東縣/1|
|17|基隆市/0|台北市/0|台北市/0|基隆市/0|基隆市/0|台北市/0|台北市/0|
|18|嘉義市/0|基隆市/0|基隆市/0|嘉義市/0|嘉義市/0|基隆市/0|基隆市/0|
|19|台東縣/0|嘉義市/0

In [16]:
#全國總交易量
total_transcation = {}
for c in list_city:
    total_transcation[c] = totalcalculate(taiwan_transacation[c])
taiwan_total_transcation = totalcalculate(total_transcation)
print("全國總交易量:")
print(taiwan_total_transcation)

#縣市總交易量排名
print("縣市總交易量排名:")
table_dict = {}
sorted_transcation = sorted(total_transcation.items(),key=lambda s:s[1], reverse=True)
for i in range(0,19):
    print("|"+str(i+1)+"|",end='')
    print(sorted_transcation[i][0]+"|"+str(sorted_transcation[i][1])+"|") 

全國總交易量:
6033
縣市總交易量排名:
|1|宜蘭縣|1530|
|2|桃園縣|799|
|3|彰化縣|713|
|4|花蓮縣|420|
|5|南投縣|356|
|6|屏東縣|328|
|7|新竹縣|325|
|8|苗栗縣|324|
|9|台中市|295|
|10|雲林縣|279|
|11|台南市|243|
|12|高雄市|174|
|13|嘉義縣|130|
|14|台東縣|60|
|15|新北市|38|
|16|新竹市|19|
|17|台北市|0|
|18|基隆市|0|
|19|嘉義市|0|
